In [25]:
import requests
import datetime
import hashlib
import json
from flask import Flask, jsonify, request
from uuid import uuid4
from urllib.parse import urlparse

In [26]:
class Earth:
    
    def __init__(self):
        self.chain = []
        self.transactions = []
        self.addBlock(proof= 1, prevHash = "0")
        self.nodes = set()
    
    def addBlock(self, proof, prevHash):
        block = {
            "index": len(self.chain),
            "timestamp": str(datetime.datetime.now()),
            "proof": proof,
            "prevHash": prevHash,
            "transactions": self.transactions
        }
        self.transactions = []
        self.chain.append(block)
        return block
    
    def getLastBlock(self):
        return self.chain[-1]
    
    def hashBlock(self, block):
        encodedBlock = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encodedBlock).hexdigest()
    
    def proofOfWork(self, prevProof):
        newProof = 1
        solved = False 
        
        while solved is False:
            hashProblem = hashlib.sha256(str(newProof ** 2 - prevProof **2).encode()).hexdigest()
            if hashProblem[:4] == "0000":
                solved = True
            else:
                newProof +=1 
        return newProof
    
    def addTransactions(self, sender, receiver, amount):
        self.transactions.append({
            "sender": sender,
            "receiver": receiver,
            "amount": amount
        })
        previousBlock = self.getLastBlock()
        return previousBlock["index"] + 1
    
    def addNode(self, address):
        nodeIP = urlparse(address)
        self.nodes.add(nodeIP.path)
    
    def isChainValid(self, chain):
        previousBlock = chain[0]
        current_index = 1
        
        while current_index < len(chain):
            currentBlock = chain[current_index]
            currentPrevHash = currentBlock["prevHash"]
            if currentPrevHash != self.hashBlock(previousBlock):
                return False
            currentProof = currentBlock["proof"]
            previousProof = previousBlock["proof"]
            hashCheck = hashlib.sha256(str(currentProof **2 - previousProof **2).encode()).hexdigest()
            if hashCheck[:4] != "0000":
                return False
            previousBlock = currentBlock
            current_index += 1
        return True
        
    
    def shouldReplaceChain(self):
        network = self.nodes
        currentChainLength = len(self.chain)
        longestChain = None
        
        for node in network:
            nodeResonse = requests.get(f"http://{node}/chain")
            if nodeResonse.status_code == 200:
                chain = nodeResonse.json()["chain"]
                length = nodeResonse.json()["length"]
                if length > currentChainLength and self.isChainValid(chain):
                    longestChain = chain
                    currentChainLength = length
            if longestChain: ### not nil we have found a longe chain 
                return True
        return False    

In [27]:
earth = Earth()

In [28]:
app = Flask(__name__)

In [29]:
nodeAddress = str(uuid4()).replace("-", "")

In [30]:
@app.route("/mine", methods = ["GET"])
def mine():
    previousBlock = earth.getLastBlock()
    previousProof = previousBlock["proof"]
    proof = earth.proofOfWork(previousProof)
    previousHash = earth.hashBlock(previousBlock)
    earth.addTransactions(sender=nodeAddress, receiver = "jason", amount=10)
    newBlock = earth.addBlock(proof, previousHash)
    response = {
        "message": "Congrats you have mined an Earth Coin",
        "index": newBlock["index"],
        "proof": newBlock["proof"],
        "timestamp": newBlock["timestamp"],
        "prevHash": newBlock["prevHash"],
    }
    return jsonify(response), 200

In [31]:
@app.route("/chain", methods = ["GET"])
def chain():
    response = {
        "chain": earth.chain,
        "length": len(earth.chain)
    }
    return jsonify(response), 200

In [32]:
@app.route("/valid", methods = ["GET"])
def valid():
    isValid = earth.isChainValid(earth.chain)
    if isValid:
        response = {
            "message": "This chain is valid build on it"
        }
    else:
         response = {
            "message": "This chain is invalid"
        }
    return jsonify(response), 200

In [33]:
@app.route("/replace", methods = ["GET"])
def replace():
    shouldReplace = earth.shouldReplaceChain()
    if shouldReplace:
        response = {
            "message": "Chain is being replaces we found a longer chain in the network",
            "chain": earth.chain,
            "length": len(earth.chain)
        }
    else:
         response = {
            "message": "Chain is currently the longest",
             "currentLength": len(earth.chain)
        }
    return jsonify(response), 200

In [34]:
@app.route("/transaction", methods = ["POST"])
def transaction():
    transactionReponse = request.get_json()
    transactionKeys = ["sender","receiver","amount"]
    if not all(key in transactionReponse for key in transactionKeys):
        return "There is a key missing", 400
    index = earth.addTransactions(sender=transactionReponse["sender"], receiver=transactionReponse["receiver"], amount=transactionReponse["amount"])
    response = {
        "message": f"your transaction will be added to block {index}"
    }
    return jsonify(response), 201

In [35]:
@app.route("/connect", methods = ["POST"])
def connect():
    nodeResponse = request.get_json()
    nodes = nodeResponse.get('nodes')
    if nodes is None:
        return "there are no nodes in this network", 400
    for node in nodes:
        earth.addNode(node)
    response = {
        "message": "All nodes have been connected",
        "nodeList": list(earth.nodes)
    }
    return jsonify(response), 201

In [ ]:
app.run(host="0.0.0.0", port=4002)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:4002/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jan/2022 19:17:41] "POST /connect HTTP/1.1" 201 -
127.0.0.1 - - [05/Jan/2022 19:18:20] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:18:20] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:18:21] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:18:25] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:18:29] "GET /replace HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:19:20] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:19:39] "GET /chain HTTP/1.1" 200 -
